# Example of generating summaries for a 10K
In this example, we will show you how to generate page summaries from a pdf using OpenAI's models via `uniflow`'s [OpenAIJsonModelFlow](https://github.com/CambioML/uniflow/blob/main/uniflow/flow/model_flow.py#L125).

For this example, we're using a [10K from Nike](https://investors.nike.com/investors/news-events-and-reports/).

### Before running the code

You will need to `uniflow` conda environment to run this notebook. You can set up the environment following the instruction: https://github.com/CambioML/uniflow/tree/main#installation.

Next, you will need a valid [OpenAI API key](https://platform.openai.com/api-keys) to run the code. Once you have the key, set it as the environment variable `OPENAI_API_KEY` within a `.env` file in the root directory of this repository. For more details, see this [instruction](https://github.com/CambioML/uniflow/tree/main#api-keys)

Finally, we are storing the Nike 10K in the `data\raw_input` directory as "nike-10k-2023.pdf". You can download the file from [here](https://s1.q4cdn.com/806093406/files/doc_downloads/2023/414759-1-_5_Nike-NPS-Combo_Form-10-K_WR.pdf).

### Update system path

In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

### Install helper packages

In [2]:
!{sys.executable} -m pip install -q langchain pandas pypdf

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


### Import dependencies

In [3]:
from dotenv import load_dotenv
import os
import pandas as pd
from uniflow.flow.client import TransformClient
from uniflow.flow.config import TransformOpenAIConfig
from uniflow.op.model.model_config import OpenAIModelConfig
from langchain.document_loaders import PyPDFLoader
from uniflow.op.prompt import Context, PromptTemplate

load_dotenv()


True

### Prepare the input data
First, we need to pre-process the PDF to get text chunks that we can feed into the model. We will use `PyPDFLoader` from langchain.

In [4]:
pdf_file = "nike-10k-2023.pdf"

##### Set current directory and input data directory.

In [5]:
dir_cur = os.getcwd()
input_file = os.path.join(f"{dir_cur}/data/raw_input/", pdf_file)

##### Load and split the pdf

In [6]:
loader = PyPDFLoader(input_file)
pages = loader.load_and_split()
page_contents = [page.page_content for page in pages]

### Prepare sample prompts

First, we need to demonstrate sample prompts for LLM. Because we are not generating the default questions and answers, we need to have a custom `instruction` and custom `examples`, which we configure in the `PromptTemplate` class.

First, we give a custom `instruction` to the `PromptTemplate`. This ensures we are instructing the LLM to generate summaries instead of the default questions and answers.

Next, we give a sample list of `Context` examples to the `PromptTemplate` class. We pass in a custom `summary` property into our `Context` objects. This is an example summary based on the `context`.

In [7]:
guided_prompt = PromptTemplate(
    instruction="Generate a one sentence summary based on the last context below. Follow the format of the examples below to include context and summary in the response",
    few_shot_prompt=[
        Context(
            context="When you're operating on the maker's schedule, meetings are a disaster. A single meeting can blow a whole afternoon, by breaking it into two pieces each too small to do anything hard in. Plus you have to remember to go to the meeting. That's no problem for someone on the manager's schedule. There's always something coming on the next hour; the only question is what. But when someone on the maker's schedule has a meeting, they have to think about it.",
            summary="Meetings disrupt the productivity of those following a maker's schedule, dividing their time into impractical segments, while those on a manager's schedule are accustomed to a continuous flow of tasks.",
        ),
    ],
)

Next, for the given `page_contents` above, we convert them to the `Context` class to be processed by `uniflow`.

In [8]:
data = [ Context(context=p[:800], summary="") for p in page_contents[6:16] if len(p) > 200 ]
data

[Context(context='We also offer interactive consumer services and experiences as well as digital products through our digital platforms, including \nfitness and activity apps; sport, fitness and wellness content; and digital services and features in retail stores that enhance the \nconsumer experience.\nSALES AND MARKETING\nWe experience moderate fluctuations in aggregate sales volume during the year. Historically, revenues in the first and fourth \nfiscal quarters have slightly exceeded those in the second and third fiscal quarters.  However, the mix of product sales may vary \nconsiderably as a result of changes in seasonal and geographic demand for particular types of footwear , apparel and equipment, \nas well as other macroeconomic, strategic, operating and logistics-related factors.\nBecause NIKE is a consume', summary=''),
 Context(context="INTERNATIONAL MARKETS\nFor fiscal 2023, non-U.S. NIKE Brand and Converse sales accounted for approximately 57% of total revenues, compared t

### Use LLM to generate data

In this example, we will use the [OpenAIModelConfig](https://github.com/CambioML/uniflow/blob/main/uniflow/model/config.py#L17)'s default LLM to generate questions and answers.

Here, we pass in our `guided_prompt` to the `TransformOpenAIConfig` to use our customized instructions and examples, instead of the `uniflow` default ones.

We also want to get the response in the `json` format instead of the `text` default, so we set the `response_format` to `json_object`.

In [9]:
config = TransformOpenAIConfig(
    prompt_template=guided_prompt,
    model_config=OpenAIModelConfig(response_format={"type": "json_object"}),
)
client = TransformClient(config)

Now we call the `run` method on the `client` object to execute the question-answer generation operation on the data shown above.

In [10]:
output = client.run(data)

  0%|          | 0/10 [00:00<?, ?it/s]

### Process the output

Let's take a look of the generated output. We need to do a little postprocessing on the raw output.

In [11]:
# Extracting context, question, and answer into a DataFrame
contexts = []
summaries = []

for item in output:
    for i in item.get('output', []):
        for response in i.get('response', []):
            if any(key not in response for key in ['context', 'summary']):
                print("Missing context or summary in response:", response)
                continue
            contexts.append(response['context'])
            summaries.append(response['summary'])

# Set display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)

df = pd.DataFrame({
    'Context': contexts,
    'Summaries': summaries,
})

styled_df = df.style.set_properties(**{'text-align': 'left'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'left')]
}])
styled_df

,Context,Summaries
0,"We also offer interactive consumer services and experiences as well as digital products through our digital platforms, including fitness and activity apps; sport, fitness and wellness content; and digital services and features in retail stores that enhance the consumer experience. SALES AND MARKETING We experience moderate fluctuations in aggregate sales volume during the year. Historically, revenues in the first and fourth fiscal quarters have slightly exceeded those in the second and third fiscal quarters. However, the mix of product sales may vary considerably as a result of changes in seasonal and geographic demand for particular types of footwear , apparel and equipment, as well as other macroeconomic, strategic, operating and logistics-related factors. Because NIKE is a consume","Nike experiences moderate fluctuations in sales volume throughout the year, with higher revenues in the first and fourth fiscal quarters, and varying mix of product sales driven by seasonal and geographic demand as well as other factors."
1,"INTERNATIONAL MARKETS For fiscal 2023, non-U.S. NIKE Brand and Converse sales accounted for approximately 57% of total revenues, compared to 60% and 61% for fiscal 2022 and fiscal 2021, respectively. We sell our products to retail accounts through our own NIKE Direct operations and through a mix of independent distributors, licensees and sales representatives around the world. W e sell to thousands of retail accounts and ship products from 67 distribution centers outside of the United States. Refer to Item 2. Properties for further information on distribution facilities outside of the United States. During fiscal 2023, NIKE's three largest customers outside of the United States accounted for approximately 14% of total non-U.S. sales. In addition to NIKE-owned and Converse-owned digita","In fiscal 2023, non-U.S. NIKE Brand and Converse sales made up 57% of total revenues, down from 60% and 61% in fiscal 2022 and fiscal 2021, with a distribution network that includes thousands of retail accounts and 67 distribution centers outside of the United States."
2,"footwear production. For fiscal 2023, factories in Vietnam, Indonesia and China manufactured approximately 50%, 27% and 18% of total NIKE Brand footwear, respectively. For fiscal 2023, four footwear contract manufacturers each accounted for greater than 10% of footwear production and in the aggregate accounted for approximately 58% of NIKE Brand footwear production. As of May 31, 2023, our contract manufacturers operated 291 finished goods apparel factories located in 31 countries. For fiscal 2023, NIKE Brand apparel finished goods were manufactured by 55 contract manufacturers, many of which operate multiple factories. The largest single finished goods apparel factory accounted for approximately 8% of total fiscal 2023 NIKE Brand apparel production. For fiscal 2023, factories in Viet","NIKE's footwear and apparel are primarily manufactured in Vietnam, Indonesia, and China, with a handful of contract manufacturers accounting for a significant portion of production."
3,"of total NIKE Brand apparel, respectively. For fiscal 2023, one apparel contract manufacturer accounted for more than 10% of apparel production, and the top five contract manufacturers in the aggregate accounted for approximately 52% of NIKE Brand apparel production. NIKE's contract manufacturers buy raw materials for the manufacturing of our footwear, apparel and equipment products. Most raw materials are available and purchased by those contract manufacturers in the countries where manufacturing takes place. The principal materials used in our footwear products are natural and synthetic rubber , plastic compounds, foam cushioning materials, natural and synthetic leather, nylon, polyester and natural fiber textiles, as well as polyurethane films used to make NIKE Air-Sole cushioning","NIKE relies on contract manufacturers for the production of its apparel and footwear, w

Finally, we can save the output to a csv file.

In [12]:
output_dir = 'data/output'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df.to_csv(f"{output_dir}/Nike_10k_Summaries.csv", index=False)